In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import nrrd
import copy

sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("Set1"))

In [2]:
patients=['FJ', 'BE', 'MJ1']
top_dir="/Volumes/ssd750/radiomics/sbrt_radiomics/data"

for patient in patients:
    print(patient)
    patient_dir = os.path.join(top_dir,patient)
    
    for dir_item in os.listdir(patient_dir):
        d = os.path.join(patient_dir, dir_item)
        
        if os.path.isdir(d):
            radiomics_dir = os.path.join(patient_dir, d, 'radiomics')
            
            for csv_item in os.listdir(radiomics_dir):
                if os.path.splitext(csv_item)[1] == '.csv':
                    df_temp = pd.read_csv(os.path.join(radiomics_dir, csv_item))
                    df_temp['timepoint'] = str(os.path.basename(dir_item))
                    df_temp['patient'] = patient
                    
                    if 'df' in locals():
                        df = pd.concat([df, df_temp])
                    else:
                        df = copy.deepcopy(df_temp)

df.reset_index(drop=True, inplace=True)

FJ
BE
MJ1


In [3]:
# Asign dummy variable for time ordering

# Assume we only have Pre and PostN where N is NOT a left-zero-padded integer
timepoints_order = list(set(df['timepoint']))
timepoints_order.remove('Pre')
timepoints_order = [int(x.replace('Post','')) for x in timepoints_order]
timepoints_order.sort()
timepoints_order = ['Post%d' % x for x in timepoints_order]
timepoints_order.insert(0,'Pre')

timepoints_map = {idx: timepoint for timepoint,idx in enumerate(timepoints_order)}
df['time_index'] = df['timepoint'].map(timepoints_map)
df.reset_index(drop=True, inplace=True)

In [19]:
fig_dir="/Volumes/ssd750/radiomics/CCR_phantom/radiomics/notebooks/20220128"

fig = plt.figure()

cols_to_plot = list(df.columns[df.columns.str.startswith("original")])
for feature in cols_to_plot:
   
    sns.relplot(data=df[(df['mask']=='Aorta_wall_JM.nrrd') | (df['mask']=='Aorta_wall_JM2.nrrd')],
                    x='time_index',
                    y=feature,
                    hue='dose_bin',
                    style='patient',
                    row='mask',
                    markers=True,
                    dashes=True,
                    sort=True,
                    kind="line")

    # "reverse" the timepoint map for xtick labels
    plt.xticks(list(timepoints_map.values()), list(timepoints_map.keys()))

    plt.xlabel("Time point")
    plt.suptitle(feature)

    plt.gcf().set_size_inches(15, 8)
    
    plt.savefig(os.path.join(fig_dir, feature + '.pdf'))    
    plt.cla()
    plt.close()

<Figure size 432x288 with 0 Axes>